In [1]:
from scripts.tools import active_dev_worksheet as adw
# import adw
import networkx as nx
from pywintypes import com_error

In [109]:
def graph_all(issues):
    g = nx.DiGraph()
    for i in issues:
        iid = i.id
        g.add_node(iid)
        pid = i.parent
        if pid is not None:
            g.add_edge(pid, iid)
    if not nx.is_forest(g):  # should not be possible
        raise ValueError("Circles in graph :(") 
    return g
     
    
def _get_versioned_roots(g, idmap):
    allroots = find_roots(g)
    fvroots = collections.defaultdict(list)
    for root in allroots:
        iss = idmap[root]
        fvn = adw.api.resource_attrib(iss.fixed_version, "name")
        fvroots[fvn].append(root)
    return fvroots
    
def find_roots(g):
    return [n for n, ind in g.in_degree() if ind == 0]
    
def _dfs_visit(g, parent, visit, depth):
    for node in sorted(g.successors(parent)):
        visit(parent, node, depth)
        _dfs_visit(g, node, visit, depth + 1)

        
def dfs_visit(g, node, visit):
    visit(None, node, 0)
    _dfs_visit(g, node, visit, 1)

    
def graph_versions(allg, idmap, versions):
    
    def visit(graph):
        def do_visit(parent, node, _depth):
            graph.add_node(node)
            if parent is not None:
                graph.add_edge(parent, node)
        return do_visit
    
    fvroots = _get_versioned_roots(g, idmap)
    graphs = {}
    for fv in versions:
        graphs[fv.name] = subg = nx.DiGraph()
        roots = fvroots.get(fv.name, ())
        for r in roots:
            dfs_visit(allg, r, visit(subg))
            
    # for un-sprinted issues, we only care about
    # ones for PBS Software
    sw_proj_id = 5
    no_sprint = fvroots.get(None, ())
    graphs['<No Sprint>'] = subg = nx.DiGraph()
    for r in no_sprint:
        iss = idmap[r]
        if iss.project.id == sw_proj_id:
            dfs_visit(allg, r, visit(subg))
        
    return graphs

In [117]:
def download(force=False):
    if "issues" in globals() and not force:
        return
    global issues, idmap, versions
    url = "https://issue.pbsbiotech.com"
    key = "7676add9cac6631410403671cdd7850311987898"
    client = adw.api.Client(url, key)

    issues = client.Issues.filter(status_id="*")
    idmap = {i.id:i for i in issues}
    versions = client.Projects.filter_versions("pbssoftware")
    versions = [v for v in versions if v.status == "open"]
download()

In [2]:
xl = adw.open_excel()
wb = xl.Workbooks.Add()
ws = wb.Worksheets(1)

In [118]:
import importlib
importlib.reload(adw.xlhelpers)
importlib.reload(adw.unionify)
importlib.reload(adw.api)
importlib.reload(adw.table)
adt = importlib.reload(adw.adtable)
adw = importlib.reload(adw)
globals().update(adt.__dict__)

In [119]:
g = graph_all(issues)
vgraphs = graph_versions(g, idmap, versions)

In [120]:
def delete_worksheets(wb):
    while wb.Worksheets.Count > 1:
        wb.Worksheets(wb.Worksheets.Count).Delete()
    ws.UsedRange.Clear()
    ws.Name = "Sheet1"
    return wb.Worksheets(1)

with adw.screen_lock(xl):
    ws = delete_worksheets(wb)
    tc = adw.TableCreator()
    for name, vg in vgraphs.items():
        if name is None:
            name = "<Uncategorized>"
        print(f"creating worksheet '{name}'")
        ws = wb.Worksheets.Add()
        ws.Name = name
        tc.create(ws, vg, idmap)
    wb.Worksheets("Sheet1").Delete()

creating worksheet 'Future Release'
creating worksheet '3.0'
creating worksheet 'Legacy'
creating worksheet '3.1.0'
creating worksheet 'IM229 C'
creating worksheet 'IC03405 J'
creating worksheet 'Active Development'
creating worksheet '<No Sprint>'


In [63]:


ws.UsedRange.Clear()

ad = adw.TableCreator()
tbl = ad.create(ws, g, idmap)

In [ ]:
sorted(issues[0].__dict__.keys())